In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
import pyarrow as pa
import pyarrow.parquet as pq

# Elaboracion de archivos para la API

Se crearon los DataFrames con la informacion justa para la implementacion de cada funcion requerida. Para cada funcion se determinaron variables criticas (ej. el parametro de entrada) que de ser nulas el registro correspondiente no tendria la iformacion minima para ser incluido en el analisis. De acuerdo con esto, se decidio eliminar estos registros.

In [2]:
games=pd.read_parquet("ETL.GAMES.parquet")

In [3]:
items=pd.read_parquet("ETL.ITEMS.parquet")

In [4]:
reviews=pd.read_parquet("ETL.REVIEWS.parquet")

In [5]:
games

,genres,item_name,release_date,year,tags,specs,price,early_access,item_id,developer
0,Action,Lost Summoner Kitty,2018-01-04,2018,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140.0,Kotoshiro
1,Free to Play,Ironbound,2018-01-04,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
2,Casual,Real Pool 3D - Poolians,2017-07-24,2017,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
3,Action,弹炸人2222,2017-12-07,2017,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400.0,彼岸领域
4,Action,Battle Royale Trainer,2018-01-04,2018,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...
27126,Action,Kebab it Up!,2018-01-04,2018,"[Action, Indie, Casual, Violent, Adventure]","[Single-player, Steam Achievements, Steam Cloud]",1.99,0.0,745400.0,Bidoniera Games
27127,Casual,Colony On Mars,2018-01-04,2018,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
27128,Casual,LOGistICAL: South Africa,2018-01-04,2018,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
27129,Indie,Russian Roads,2018-01-04,2018,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich


In [6]:
items

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277,10.0,Counter-Strike,0.100000
1,76561197970982479,277,20.0,Team Fortress Classic,0.000000
2,76561197970982479,277,30.0,Day of Defeat,0.116667
3,76561197970982479,277,40.0,Deathmatch Classic,0.000000
4,76561197970982479,277,50.0,Half-Life: Opposing Force,0.000000
...,...,...,...,...,...
5153204,76561198329548331,7,346330.0,BrainBread 2,0.000000
5153205,76561198329548331,7,373330.0,All Is Dust,0.000000
5153206,76561198329548331,7,388490.0,One Way To Die: Steam Edition,0.050000
5153207,76561198329548331,7,521570.0,You Have 10 Seconds 2,0.066667


In [7]:
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,Euro Truck Simulator 2
...,...,...,...,...,...
52753,76561198312638244,130.0,True,if you liked Half life i would really recommen...,Half-Life: Blue Shift
52754,76561198312638244,70.0,True,a must have classic from steam definitely wort...,Half-Life
52755,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,Black Mesa
52756,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,Counter-Strike Nexon: Zombies


FUNCIONES DE CONSULTA

### def developer( desarrollador:str)

Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. <br><br>


Columnas necesarias:
- year (games)
- price (games)
- item_id (games)
- developer (games)

In [8]:
def_developer = games[['year','price','item_id','developer']]
def_developer.drop(def_developer[def_developer['developer'].isnull()].index,inplace=True)
def_developer

C:\Users\Usuario\AppData\Local\Temp\ipykernel_23972\3230172358.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def_developer.drop(def_developer[def_developer['developer'].isnull()].index,inplace=True)


,year,price,item_id,developer
0,2018,4.99,761140.0,Kotoshiro
1,2018,0.00,643980.0,Secret Level SRL
2,2017,0.00,670290.0,Poolians.com
3,2017,0.99,767400.0,彼岸领域
4,2018,3.99,772540.0,Trickjump Games Ltd
...,...,...,...,...
27126,2018,1.99,745400.0,Bidoniera Games
27127,2018,1.99,773640.0,"Nikita ""Ghost_RUS"""
27128,2018,4.99,733530.0,Sacada
27129,2018,1.99,610660.0,Laush Dmitriy Sergeevich


In [9]:
def_developer.isnull().sum()

year         0
price        0
item_id      0
developer    0
dtype: int64

### def UserForGenre( genero: str)

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Columnas necesarias:
- user_id (items)
- playtime_forever (items)
- item_id (items, games)
- genres (games)
- year (games)

In [10]:
def_userforgenre = items[['user_id','playtime_forever','item_id']]
def_userforgenre = def_userforgenre.merge(games[['item_id','genres','year']],on='item_id')
def_userforgenre.drop(def_userforgenre[def_userforgenre['genres'].isnull()].index,inplace=True)
def_userforgenre

,user_id,playtime_forever,item_id,genres,year
0,76561197970982479,0.100000,10.0,Action,2000
1,76561197970982479,0.000000,20.0,Action,1999
2,76561197970982479,0.116667,30.0,Action,2003
3,76561197970982479,0.000000,40.0,Action,2001
4,76561197970982479,0.000000,50.0,Action,1999
...,...,...,...,...,...
4065128,76561198329548331,11.283333,304930.0,Action,2017
4065129,76561198329548331,0.716667,227940.0,Action,2016
4065130,76561198329548331,0.000000,346330.0,Action,2016
4065131,76561198329548331,0.050000,388490.0,Adventure,2015


In [11]:
def_userforgenre.isnull().sum()

user_id             0
playtime_forever    0
item_id             0
genres              0
year                0
dtype: int64

### Exportación de archivos a formato .parquet

In [13]:
pq.write_table(pa.Table.from_pandas(def_developer),'def_developer.parquet')
pq.write_table(pa.Table.from_pandas(def_userforgenre),'def_userforgenre.parquet')